In [1]:
import os
import pandas as pd
from google.cloud import bigquery
from google.oauth2.service_account import Credentials
import db_dtypes

In [2]:
BIGQUERY_CREDENTIALS_FILE_PATH = r"C:\TIR101_Group2\secrets\harry_GCS_BigQuery_write_cred.json"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = BIGQUERY_CREDENTIALS_FILE_PATH

In [3]:
BQ_CLIENT = bigquery.Client()  


In [4]:
def etl_sales_src_to_bq(client: bigquery.Client = BQ_CLIENT) -> None:
    query_job = client.query(
        """
        CREATE OR REPLACE EXTERNAL TABLE `bike_v1.youbike_ods_test`
        WITH PARTITION COLUMNS
        OPTIONS (
        format = 'CSV',
        uris = ['gs://youbike_realtime/*.csv'],
        hive_partition_uri_prefix = 'gs://youbike_realtime/',
        skip_leading_rows = 1,
        max_bad_records = 1
        );
        """
    )

    query_job.result()

    print("youbike updated.")

In [5]:
etl_sales_src_to_bq()

youbike updated.


In [6]:
def query_stackoverflow(client: bigquery.Client = BQ_CLIENT):
    query_job = client.query(
        """
        SELECT *
        FROM `bike_v1.youbike_ods_test`
        """
    )
    return query_job.to_dataframe()  # Convert result to DataFrame

df = query_stackoverflow()


In [7]:
df

,sno,sna,sarea,mday,ar,sareaen,snaen,aren,act,srcUpdateTime,updateTime,infoTime,infoDate,total,available_rent_bikes,latitude,longitude,available_return_bikes,dt
0,500101004,YouBike2.0_和平公園東側,大安區,2024-05-07 20:28:20+00:00,和平東路二段118巷33號,Daan Dist.,YouBike2.0_Heping Park (East),No. 33， Ln. 118， Sec. 2， Heping E. Rd,1,2024-05-07 20:28:25+00:00,2024-05-07 20:28:51+00:00,2024-05-07 20:28:20+00:00,2024-05-07,11,1,25.023510,121.542820,10,05_07_2024
1,500101010,YouBike2.0_新生南路三段82號前,大安區,2024-05-07 20:21:19+00:00,新生南路三段82號,Daan Dist.,YouBike2.0_No. 82， Sec. 3， Xinsheng S. Rd.,No. 82， Sec. 3， Xinsheng S. Rd.,1,2024-05-07 20:28:25+00:00,2024-05-07 20:28:51+00:00,2024-05-07 20:21:19+00:00,2024-05-07,20,8,25.018940,121.533610,12,05_07_2024
2,500101016,YouBike2.0_羅斯福路三段245號前,大安區,2024-05-07 20:28:14+00:00,羅斯福路三段245號,Daan Dist.,YouBike2.0_No. 245， Sec. 3， Roosevelt Rd.,No. 245， Sec. 3， Roosevelt Rd.,1,2024-05-07 20:28:25+00:00,2024-05-07 20:28:51+00:00,2024-05-07 20:28:14+00:00,2024-05-07,8,8,25.019270,121.529890,0,05_07_2024
3,500101018,YouBike2.0_溫州公園,大安區,2024-05-07 20:28:19+00:00,羅斯福路三段283巷11號,Daan Dist.,YouBike2.0_Wenzhou Park,No. 11， Ln. 283， Sec. 3， Roosevelt Rd.,1,2024-05-07 20:28:25+00:00,2024-05-07 20:28:51+00:00,2024-05-07 20:28:19+00:00,2024-05-07,5,2,25.018950,121.531560,3,05_07_2024
4,500101019,YouBike2.0_和平臥龍街口,大安區,2024-05-07 20:19:14+00:00,臥龍街1號,Daan Dist.,YouBike2.0_Heping Wolong St. Intersection,No. 1， Wolong St.,1,2024-05-07 20:28:25+00:00,2024-05-07 20:28:51+00:00,2024-05-07 20:19:14+00:00,2024-05-07,15,1,25.024461,121.546705,14,05_07_2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1025833,500119064,YouBike2.0_臺大明達館機車停車場,臺大公館校區,2024-05-12 11:39:14+00:00,臺大明達館西側,NTU Dist,YouBike2.0_NTU Ming - Da Hall Parking,NTU Ming - Da Hall(West),1,2024-05-12 11:48:25+00:00,2024-05-12 11:48:51+00:00,2024-05-12 11:39:14+00:00,2024-05-12,28,0,25.017540,121.543980,28,05_12_2024
1025834,500119074,YouBike2.0_臺大生命科學館西北側,臺大公館校區,2024-05-12 11:46:20+00:00,臺大動物博物館西側,NTU Dist,YouBike2.0_NTU Life Science Building(Northwest),Museum of Zoology(West),1,2024-05-12 11:48:25+00:00,2024-05-12 11:48:51+00:00,2024-05-12 11:46:20+00:00,2024-05-12,30,6,25.015480,121.538250,24,05_12_2024
1025835,500119087,YouBike2.0_臺大總圖書館西南側,臺大公館校區,2024-05-12 11:48:19+00:00,臺大圖書館西南側,NTU Dist,YouBike2.0_NTU Main Library(Southwest),NTU Main Library(Southwest),1,2024-05-12 11:48:25+00:00,2024-05-12 11:48:51+00:00,2024-05-12 11:48:19+00:00,2024-05-12,30,4,25.016900,121.540310,16,05_12_2024
1025836,500119088,YouBike2.0_臺大黑森林西側,臺大公館校區,2024-05-12 11:41:15+00:00,臺大霖澤館南側,NTU Dist,YouBike2.0_NTU Black Forest(West),NTU Tsai Lecture Hall(South),1,2024-05-12 11:48:25+00:00,2024-05-12 11:48:51+00:00,2024-05-12 11:41:15+00:00,2024-05-12,20,1,25.019950,121.543470,19,05_12_2024
